<a href="https://colab.research.google.com/github/ProfDoof/advent_of_code/blob/2022/day14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 

## Load Data

In [44]:
from pathlib import Path

DAY = 14
DATA_FILE = Path.cwd() / 'drive' / 'MyDrive' / 'AdventOfCode' / 'aoc_data' / f'day{DAY}.txt'

data = DATA_FILE.read_text()

## Solution

In [ ]:
test = '''
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
'''

In [47]:
from typing import NamedTuple, List, Dict, Tuple
import itertools
import math
from functools import partial
from collections import defaultdict

class Point(NamedTuple):
    x: int
    y: int

class RockPath(NamedTuple):
    points: List[Point]

    def __iter__(self):
        start_iter, end_iter = itertools.tee(self.points)
        next(end_iter, None)
        for start_point, end_point in zip(start_iter, end_iter):
            x_iter = itertools.repeat(start_point.x)
            y_iter = itertools.repeat(start_point.y)
            if start_point.x != end_point.x:
                direction = int(math.copysign(1, -start_point.x + end_point.x))
                x_iter = range(start_point.x, end_point.x, direction)
            else:
                direction = int(math.copysign(1, -start_point.y + end_point.y))
                y_iter = range(start_point.y, end_point.y, direction)
            
            yield from ((x, y) for x, y in zip(x_iter, y_iter))
        x_final, y_final = self.points[-1]
        yield x_final, y_final


def get_rock_path(i_data: str):
    return RockPath([Point(int(x), int(y)) for x, y in map(lambda x: x.split(','), i_data.split(' -> '))])


def get_air():
    return '.'


def get_floor():
    return '#'


class Map:
    def __init__(self, map: List[Dict[int, str]]):
        self.map = map
        self.map.append(defaultdict(get_air))
        self.map.append(defaultdict(get_floor))
        self.changed = True

    def __str__(self):
        return '\n'.join(' '.join(row.values()) for row in self.map)

    def drop(self):
        c_x, c_y = (500, 0)
        moved = True
        while moved:
            moved = False
            p_y = c_y + 1
            for p_x in [c_x, c_x - 1, c_x + 1]:
                if self.map[p_y][p_x] == '.':
                    moved = True
                    self.map[p_y][p_x] = 'o'
                    self.map[c_y][c_x] = '.' if self.map[c_y][c_x] == 'o' else '+'
                    c_x, c_y = p_x, p_y
                    break

        if (c_x, c_y) == (500, 0):
            self.map[c_y][c_x] = 'o'
            self.changed = False
        
        return self



def get_map(i_data: str):
    sand_drops = [(500, 0)]
    rock_points = []
    for path_data in i_data.strip().split('\n'):
        path = get_rock_path(path_data)
        rock_points.extend(path)

    min_x, min_y = math.inf, math.inf
    max_x, max_y = 0, 0
    for x, y in itertools.chain(sand_drops, rock_points):
        min_x, min_y = min(min_x, x), min(min_y, y)
        max_x, max_y = max(max_x, x), max(max_y, y)

    map = [defaultdict(get_air) for i in range(min_y, max_y + 1)]
    for sd_x, sd_y in sand_drops:
        map[sd_y][sd_x] = '+'
    
    for rp_x, rp_y in rock_points:
        map[rp_y][rp_x] = '#'

    return Map(map)

the_map = get_map(data)

while the_map.changed:
    the_map.drop()

units_of_sand = 0

for row in the_map.map:
    for val in row.values():
        if val == 'o':
            units_of_sand += 1

print(units_of_sand)

28594
